In [1]:
import json
import numpy as np
import math
import cv2
import mediapipe as mp
from tensorflow.keras.models import load_model

In [2]:
# Define angle thresholds for each pose
ANGLE_THRESHOLDS = {
    'downdog': {
        'left_elbow_angle': (150, 200),
        'right_elbow_angle': (150, 200),
        'left_shoulder_angle': (150, 220),
        'right_shoulder_angle': (150, 220),
        'left_knee_angle': (140, 200),
        'right_knee_angle': (140, 200),
        'left_hip_angle': (60, 110),
        'right_hip_angle': (60, 110)
    },
    'goddess': {
        'left_elbow_angle': (70, 120),
        'right_elbow_angle': (70, 120),
        'left_shoulder_angle': (70, 110),
        'right_shoulder_angle': (70, 110),
        'left_knee_angle': (90, 140),
        'right_knee_angle': (90, 140),
        'left_hip_angle': (40, 90),
        'right_hip_angle': (40, 90)
    },
    'plank': {
        'left_elbow_angle': (160, 180),
        'right_elbow_angle': (160, 180),
        'left_shoulder_angle': (70, 120),
        'right_shoulder_angle': (70, 120),
        'left_knee_angle': (160, 180),
        'right_knee_angle': (160, 180),
        'left_hip_angle': (150, 180),
        'right_hip_angle': (150, 180)
    },
    'warrior2': {
        'left_elbow_angle': (160, 180),
        'right_elbow_angle': (160, 180),
        'left_shoulder_angle': (80, 120),
        'right_shoulder_angle': (80, 120),
        'left_knee_angle': (80, 120),
        'right_knee_angle': (80, 120),
        'left_hip_angle': (80, 120),
        'right_hip_angle': (80, 120)
    },
    'tree': {
        'left_elbow_angle': (160, 180),
        'right_elbow_angle': (160, 180),
        'left_shoulder_angle': (70, 120),
        'right_shoulder_angle': (70, 120),
        'left_knee_angle': (150, 180),
        'right_knee_angle': (150, 180),
        'left_hip_angle': (80, 120),
        'right_hip_angle': (80, 120)
    }
}

In [3]:
# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    angle = math.degrees(math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0]))
    angle = abs(angle)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [4]:
# Function to extract joint coordinates from the image
def extract_angles_from_image(image_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()
    
    # Load and process image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        # Get coordinates for key joints
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

        # Calculate angles
        angles = {
            'left_elbow_angle': calculate_angle(left_shoulder, left_elbow, left_wrist),
            'right_elbow_angle': calculate_angle(right_shoulder, right_elbow, right_wrist),
            'left_shoulder_angle': calculate_angle(left_elbow, left_shoulder, left_hip),
            'right_shoulder_angle': calculate_angle(right_elbow, right_shoulder, right_hip),
            'left_knee_angle': calculate_angle(left_hip, left_knee, left_ankle),
            'right_knee_angle': calculate_angle(right_hip, right_knee, right_ankle),
            'left_hip_angle': calculate_angle(left_shoulder, left_hip, left_knee),
            'right_hip_angle': calculate_angle(right_shoulder, right_hip, right_knee)
        }
        return angles
    else:
        return None

In [5]:
# Load LSTM model
model = load_model('verification_model_implementation_2.h5')

In [36]:
# Function to predict pose correctness with a pose label
def predict_pose_correctness(image_path, pose_label):
    angles = extract_angles_from_image(image_path)
    if angles is None:
        return f"No pose landmarks detected for {pose_label}."

    # Reshape for LSTM model
    angles_array = np.array(list(angles.values())).reshape(1, 1, len(angles))

    # Predict using the model
    prediction = model.predict(angles_array)
    correctness = prediction[0][0] > 0.5  # Binary classification threshold

    if correctness:
        return "yes"
    else:
        incorrect_parts = []
        for angle_name, angle_value in angles.items():
            if angle_name in ANGLE_THRESHOLDS:
                min_threshold, max_threshold = ANGLE_THRESHOLDS[angle_name]
                if not (min_threshold <= angle_value <= max_threshold):
                    incorrect_parts.append(f"{angle_name} is out of range ({angle_value:.2f}°)")

        if incorrect_parts:
            return f"No, issues found: {', '.join(incorrect_parts)}"
        else:
            return "No, but no specific angle issues detected."

In [37]:
# Example usage
image_path = 'C:/Users/HP/Desktop/test_12.jfif'
pose_label = 'Goddess Pose'
result = predict_pose_correctness(image_path, pose_label)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
No, but no specific angle issues detected.
